<a href="https://colab.research.google.com/github/intelligent-environments-lab/CityLearn/blob/master/examples/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QuickStart

Install the latest CityLearn version from PyPi with the :code:`pip` command:

In [ ]:
pip install CityLearn

## Centralized RBC
Run the following to simulate an environment controlled by centralized RBC agent for a single episode:

In [1]:
from citylearn.citylearn import CityLearnEnv
from citylearn.agents.rbc import BasicRBC as RBCAgent

dataset_name = 'citylearn_challenge_2022_phase_1'
env = CityLearnEnv(dataset_name, central_agent=True)
model = RBCAgent(env)
model.learn(episodes=1)

# print cost functions at the end of episode
for n, nd in env.evaluate().groupby('name'):
    nd = nd.pivot(index='name', columns='cost_function', values='value').round(3)
    print(n, ':', nd.to_dict('records'))

Building_1 : [{'carbon_emissions': 1.134, 'cost': 1.043, 'electricity_consumption': 1.184, 'zero_net_energy': 1.118}]
Building_2 : [{'carbon_emissions': 1.158, 'cost': 1.063, 'electricity_consumption': 1.215, 'zero_net_energy': 1.101}]
Building_3 : [{'carbon_emissions': 1.272, 'cost': 1.145, 'electricity_consumption': 1.346, 'zero_net_energy': 1.294}]
Building_4 : [{'carbon_emissions': 1.181, 'cost': 1.097, 'electricity_consumption': 1.237, 'zero_net_energy': 1.085}]
Building_5 : [{'carbon_emissions': 1.186, 'cost': 1.075, 'electricity_consumption': 1.262, 'zero_net_energy': 1.145}]
District : [{'1 - load_factor': 0.987, 'average_daily_peak': 1.15, 'carbon_emissions': 1.186, 'cost': 1.085, 'electricity_consumption': 1.249, 'peak_demand': 1.052, 'ramping': 1.162, 'zero_net_energy': 1.148}]


## Decentralized-Independent SAC

Run the following to simulate an environment controlled by decentralized-independent SAC agents for 1 training episode:

In [2]:
from citylearn.citylearn import CityLearnEnv
from citylearn.agents.sac import SAC as RLAgent

dataset_name = 'citylearn_challenge_2022_phase_1'
env = CityLearnEnv(dataset_name)
model = RLAgent(env)
model.learn(episodes=2, deterministic_finish=True)

for n, nd in env.evaluate().groupby('name'):
    nd = nd.pivot(index='name', columns='cost_function', values='value').round(3)
    print(n, ':', nd.to_dict('records'))

Building_1 : [{'carbon_emissions': 0.953, 'cost': 0.944, 'electricity_consumption': 0.951, 'zero_net_energy': 1.079}]
Building_2 : [{'carbon_emissions': 0.989, 'cost': 0.985, 'electricity_consumption': 0.987, 'zero_net_energy': 1.056}]
Building_3 : [{'carbon_emissions': 1.007, 'cost': 0.985, 'electricity_consumption': 0.989, 'zero_net_energy': 1.181}]
Building_4 : [{'carbon_emissions': 1.04, 'cost': 1.027, 'electricity_consumption': 1.024, 'zero_net_energy': 1.045}]
Building_5 : [{'carbon_emissions': 0.975, 'cost': 0.926, 'electricity_consumption': 0.983, 'zero_net_energy': 1.063}]
District : [{'1 - load_factor': 0.994, 'average_daily_peak': 0.951, 'carbon_emissions': 0.993, 'cost': 0.974, 'electricity_consumption': 0.987, 'peak_demand': 0.943, 'ramping': 1.049, 'zero_net_energy': 1.085}]


## Decentralized-Cooperative MARLISA

Run the following to simulate an environment controlled by decentralized-cooperative MARLISA agents for 1 training episode:

In [3]:
from citylearn.citylearn import CityLearnEnv
from citylearn.agents.marlisa import MARLISA as RLAgent

dataset_name = 'citylearn_challenge_2022_phase_1'
env = CityLearnEnv(dataset_name)
model = RLAgent(env)
model.learn(episodes=2, deterministic_finish=True)

for n, nd in env.evaluate().groupby('name'):
    nd = nd.pivot(index='name', columns='cost_function', values='value').round(3)
    print(n, ':', nd.to_dict('records'))

Building_1 : [{'carbon_emissions': 1.025, 'cost': 1.016, 'electricity_consumption': 1.03, 'zero_net_energy': 1.019}]
Building_2 : [{'carbon_emissions': 1.0, 'cost': 1.0, 'electricity_consumption': 1.0, 'zero_net_energy': 1.0}]
Building_3 : [{'carbon_emissions': 0.998, 'cost': 0.995, 'electricity_consumption': 0.999, 'zero_net_energy': 1.016}]
Building_4 : [{'carbon_emissions': 1.0, 'cost': 0.999, 'electricity_consumption': 1.0, 'zero_net_energy': 1.002}]
Building_5 : [{'carbon_emissions': 1.0, 'cost': 1.0, 'electricity_consumption': 1.0, 'zero_net_energy': 1.0}]
District : [{'1 - load_factor': 1.0, 'average_daily_peak': 1.002, 'carbon_emissions': 1.005, 'cost': 1.002, 'electricity_consumption': 1.006, 'peak_demand': 1.0, 'ramping': 1.016, 'zero_net_energy': 1.007}]


# Stable Baselines3 Reinforcement Learning Algorithms

Install the latest version of Stable Baselines3:

In [ ]:
pip install stable-baselines3

Before the environment is ready for use in Stable Baselines3, it needs to be wrapped. Firstly, wrap the environment using the `NormalizedObservationWrapper` (see [docs](https://www.citylearn.net/api/citylearn.wrappers.html#citylearn.wrappers.NormalizedObservationWrapper)) to ensure that observations served to the agent are min-max normalized between [0, 1] and cyclical observations e.g. hour, are encoded using the cosine transformation.

Next, we wrap with the `StableBaselines3Wrapper` (see [docs](https://www.citylearn.net/api/citylearn.wrappers.html#citylearn.wrappers.StableBaselines3Wrapper)) that ensures observations, actions and rewards are served in manner that is compatible with Stable Baselines3 interface.

> ⚠️ **NOTE**: `central_agent` in the `env` must be `True` when using Stable Baselines3  as it does not support multi-agents.

In [4]:
from stable_baselines3.sac import SAC
from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

dataset_name = 'citylearn_challenge_2022_phase_1'
env = CityLearnEnv(dataset_name, central_agent=True)
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)
model = SAC('MlpPolicy', env)
model.learn(total_timesteps=env.time_steps*2)

observations = env.reset()

while not env.done:
    actions, _ = model.predict(observations, deterministic=True)
    observations, _, _, _ = env.step(actions)

for n, nd in env.evaluate().groupby('name'):
    nd = nd.pivot(index='name', columns='cost_function', values='value').round(3)
    print(n, ':', nd.to_dict('records'))

Building_1 : [{'carbon_emissions': 1.107, 'cost': 1.104, 'electricity_consumption': 1.08, 'zero_net_energy': 1.129}]
Building_2 : [{'carbon_emissions': 1.078, 'cost': 0.998, 'electricity_consumption': 1.075, 'zero_net_energy': 1.087}]
Building_3 : [{'carbon_emissions': 1.211, 'cost': 1.191, 'electricity_consumption': 1.191, 'zero_net_energy': 1.235}]
Building_4 : [{'carbon_emissions': 1.269, 'cost': 1.402, 'electricity_consumption': 1.204, 'zero_net_energy': 1.087}]
Building_5 : [{'carbon_emissions': 1.273, 'cost': 1.205, 'electricity_consumption': 1.277, 'zero_net_energy': 1.12}]
District : [{'1 - load_factor': 1.012, 'average_daily_peak': 1.106, 'carbon_emissions': 1.188, 'cost': 1.18, 'electricity_consumption': 1.165, 'peak_demand': 1.261, 'ramping': 1.45, 'zero_net_energy': 1.132}]
